In [1]:
import requests
import json

def fetch_and_categorize_papers(venue_id):
    url = f"https://api2.openreview.net/notes?content.venueid={venue_id}"
    response = requests.get(url)
    data = response.json()
    
    # Initialize dictionaries to hold categorized papers
    papers_by_type = {'Oral': [], 'Spotlight': [], 'Poster': []}
    
    # Iterate over all papers and categorize them
    if 'notes' in data:
        for note in data['notes']:
            venue_info = note['content'].get('venue', {})
            if 'value' in venue_info:
                venue_value = venue_info['value']
                if 'oral' in venue_value.lower():
                    papers_by_type['Oral'].append(note)
                elif 'spotlight' in venue_value.lower():
                    papers_by_type['Spotlight'].append(note)
                elif 'poster' in venue_value.lower():
                    papers_by_type['Poster'].append(note)
    
    return papers_by_type


def filter_paper_by_topic(venue_id, topic, papers_by_type=None):
    if not papers_by_type:
        fetch_and_categorize_papers(venue_id)
    # Print the results to verify
    # for paper_type, notes in papers_by_type.items():
    #     print(f"\n{paper_type} Papers:")
    #     if notes:
    #         for note in notes[:5]:  # Limiting to first 5 papers for brevity
    #             title = note.get('content', {}).get('title', 'No title available')
    #             authors = ", ".join(note.get('content', {}).get('authors', []))
    #             abstract = note.get('content', {}).get('abstract', 'No abstract available')
    #             tldr = note.get('content', {}).get('TLDR', 'No TL;DR available')
    #             keywords = note.get('content', {}).get('keywords', 'No keywords available')
    #             link = f"https://openreview.net/forum?id={note['id']}"
    #             print(f"Title: {title}")
    #             print(f"Authors: {authors}")
    #             print(f"Abstract: {abstract}")
    #             print(f"TL;DR: {tldr}")
    #             print(f"Keywords: {keywords}")
    #             print(f"Link: {link}")
    #             print("---")
    #     else:
    #         print("No papers found.")

    for paper_type, notes in papers_by_type.items():
        print(f"\n{paper_type} Papers:")
        if notes:
            for note in notes: 
                title = note.get('content', {}).get('title', 'No title available').get('value')
                authors = ", ".join(note.get('content', {}).get('authors', []).get('value'))
                abstract = note.get('content', {}).get('abstract', 'No abstract available').get('value')

                tldr = note.get('content', {}).get('TLDR', '')
                if tldr:
                    tldr = tldr['value']
                else:
                    tldr = 'No TL;DR available'

                keywords = note.get('content', {}).get('keywords', 'No keywords available').get('value')
                keywords = '; '.join(keywords)

                if (topic in tldr.lower()) or (topic in keywords.lower()) or (topic in abstract.lower()) or (topic in title.lower()):
                    # print(f"Reasoning Paper")

                    # link = f"https://openreview.net/forum?id={note['id']}"
                    print(f"Title: {title}")
                    print(f"Authors: {authors}")
                    # print(f"Abstract: {abstract}")
                    print(f"TL;DR: {tldr}")
                    print(f"Keywords: {keywords}")
                    # print(f"Link: {link}")
                    print("---")


In [2]:
import json

venue = 'ICLR' # NeurIPS
year = 2024 # 2023
venue_id = f"{venue}.cc/{year}/Conference"
# venue_id = "NeurIPS.cc/2023/Conference"

papers_by_type = fetch_and_categorize_papers(venue_id)

with open(f'res/{venue}{year}.json', 'w+') as f:
    json.dump(papers_by_type, f)

In [3]:
# with open(f'res/{venue}{year}.json', 'w+') as f:
#     papers_by_type = json.load(f)

filter_paper_by_topic(venue_id, 'reasoning', papers_by_type)


Oral Papers:
Title: Phenomenal Yet Puzzling: Testing Inductive Reasoning Capabilities of Language Models with Hypothesis Refinement
Authors: Linlu Qiu, Liwei Jiang, Ximing Lu, Melanie Sclar, Valentina Pyatkin, Chandra Bhagavatula, Bailin Wang, Yoon Kim, Yejin Choi, Nouha Dziri, Xiang Ren
TL;DR: No TL;DR available
Keywords: language model; natural language processing; inductive reasoning
---
Title: MathVista: Evaluating Mathematical Reasoning of Foundation Models in Visual Contexts
Authors: Pan Lu, Hritik Bansal, Tony Xia, Jiacheng Liu, Chunyuan Li, Hannaneh Hajishirzi, Hao Cheng, Kai-Wei Chang, Michel Galley, Jianfeng Gao
TL;DR: We introduce MathVista, a novel benchmark for evaluating mathematical reasoning capabilities within visual contexts, and conduct extensive experiments on 11 foundation models.
Keywords: large language models; large multimodal models; mathematical reasoning; vision-language reasoning; foundation models and their evaluations
---
Title: SWE-bench: Can Language Mo